In [1]:
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

In [2]:
from googleapiclient.discovery import build
def pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=DATA_TO_PULL).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=DATA_TO_PULL).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data


In [3]:
import pandas as pd
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '1DCHfy9S5AZmtE-cCwkRyMWxh0iAtEz5Lgw6o6QSqFdE'
DATA_TO_PULL = 'Form Responses'
data = pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL)
df = pd.DataFrame(data[1:], columns = data[0])
df

RefreshError: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})

In [ ]:
import numpy as np
df=df.replace("", np.nan)


In [ ]:
df['Submission Date'] = pd.to_datetime(df['Submission Date'])
df = df.sort_values(by=['Submission Date'])

# remove duplicates, keeping last entry (most recent entry)
df = df.drop_duplicates(subset=['First Name', 'Last Name'], keep='last')
df

In [ ]:
columns = ['Submission Date', 'Name', 'Yes', 'No', 'Comments or Questions']
out = pd.DataFrame(columns=columns)

In [ ]:
cnt = 0
# loop over each submission
for i, row in df.iterrows():
    submitter = str(row[1]).title() + " " + str(row[2]).title()
    for col in columns:
        if col in row:
            out.loc[cnt, col] = row[col]
        if row[4] == 'Yes':
            out.loc[cnt, 'Yes'] = row[3]
        else:
            out.loc[cnt, 'No'] = row[3]
    out.loc[cnt, 'Name'] = submitter
    cnt +=1
out

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import json
scopes = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive'
]
credentials = ServiceAccountCredentials.from_json_keyfile_name("rsvp-365109-be34f987e91c.json", scopes) #access the json key you downloaded earlier 
file = gspread.authorize(credentials) # authenticate the JSON key with gspread
sheet = file.open("JustineXMin")  #open sheet


In [ ]:
# get the first sheet of the Spreadsheet
sheet_instance = sheet.get_worksheet(1)
out.fillna(value="")
out

In [ ]:
sheet_instance.update_acell('A1', 'WARNING DO NOT EDIT: THIS IS AUTOMATICALLY GENERATED')

In [ ]:
out['Yes'] = out['Yes'].fillna(0).astype(int)
yes_total = out['Yes'].sum()
out['No'] = out['No'].fillna(0).astype(int)
no_total = out['No'].sum()
out = out.fillna(value="")
out['Submission Date'] = out['Submission Date'].astype(str)
sheet_instance.update('A6', out.values.tolist())

In [ ]:
 # add totals to sheet
sheet_instance.update_acell('B3', 'Yes')
sheet_instance.update_acell('C3', 'No')
sheet_instance.update_acell('A4', 'Totals')
sheet_instance.update_acell('B4', '%d' %yes_total)
sheet_instance.update_acell('C4', '%d' %no_total)

In [ ]:
import pandas as pd
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '1DCHfy9S5AZmtE-cCwkRyMWxh0iAtEz5Lgw6o6QSqFdE'
DATA_TO_PULL = 'ExpectedVsActual'
data = pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL)
df = pd.DataFrame(data).T
df = df[:8].T
df = df[1:]
df[0] = df[[0, 1]].apply(" ".join, axis=1)
df

In [ ]:
invite_first = df[0].apply(lambda x: x.lower().replace(" ", ""))
print(out)

In [ ]:
ws = sheet.get_worksheet(2)
ws

In [ ]:
for i, row in out.iterrows():
    index = invite_first == row['Name'].lower().replace(" ", "")
    if index.sum()==1:      
        cellans = (str('G')+str(2 + np.where(df[0]==row['Name'])[0][0]))
        cellkawn = (str('H')+str(2 + np.where(df[0]==row['Name'])[0][0]))
        ans = 'Yes' if row['Yes'] != 0 else 'No'
        kawnt = row['Yes'] if row['Yes'] != 0 else row['No']
        ws.update_acell(cellans, ans)
        ws.update_acell(cellkawn, kawnt)
    elif index.sum() == 0:
        print("warning: %s submission has no match in invite list" %(row['Name']))

    # multiple matches!
    else:
        print("warning: %s has multiple match on invite list" %(row['Name']))


In [ ]:
yes_total = ws.acell('M6').value
no_total = ws.acell('N6').value
print("Total YES RSVPS: %s" %yes_total)
print("Total NO RSVPS: %s" %no_total)